In [1]:
import sys
import pickle
import torch
import os
import numpy as np
import mxnet as mx
from mxnet import ndarray as nd

sys.path.append("/app/sandbox/AdaFace")
from AdaFace.data import test_dataset, prepare_data
from AdaFace.evaluate_utils import get_val_data

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [2]:
def load_bin(path, image_size=[112, 112]):
    try:
        with open(path, "rb") as f:
            bins, issame_list = pickle.load(f)  # py2
    except UnicodeDecodeError as e:
        with open(path, "rb") as f:
            bins, issame_list = pickle.load(f, encoding="bytes")  # py3
    data_list = []
    for flip in [0, 1]:
        data = torch.empty((len(issame_list) * 2, 3, image_size[0], image_size[1]))
        data_list.append(data)
    for idx in range(len(issame_list) * 2):
        _bin = bins[idx]
        img = mx.image.imdecode(_bin)
        if img.shape[1] != image_size[0]:
            img = mx.image.resize_short(img, image_size[0])
        img = nd.transpose(img, axes=(2, 0, 1))
        for flip in [0, 1]:
            if flip == 1:
                img = mx.ndarray.flip(data=img, axis=2)
            data_list[flip][idx][:] = torch.from_numpy(img.asnumpy())
        if idx % 1000 == 0:
            print("loading bin", idx)
    print(data_list[0].shape)
    return data_list, issame_list

In [3]:
# data_set = load_bin('/app/datasets/ms1m/agedb_30.bin')
# len(data_set[1]), np.sum(data_set[1])
# data_set[0][0].shape

In [6]:
%pwd
%cd /app

/app


In [8]:
data_root = "/app/datasets"
val_data_path = "ms1m"
five_ds = test_dataset(data_root, val_data_path, "five_mem_concat")

laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile


In [9]:
from pathlib import Path
from tqdm import tqdm

# save metadata
out_dir = Path("/app/cache/five_ds_embs")
all_issame = []
dataname = []
for i in tqdm(range(len(five_ds))):
    all_issame.append(five_ds[i][1])
    dataname.append(five_ds[i][2])
all_issame = np.array(all_issame)
dataname = np.array(dataname)
np.save(out_dir / "all_issame.npy", all_issame)
np.save(out_dir / "dataname.npy", dataname)

100%|██████████| 62000/62000 [00:16<00:00, 3698.50it/s]


### Compute metrics

In [10]:
emb_dir = Path("/app/cache/five_ds_embs")
pred = np.load(emb_dir / "default_scf_embs_five_ds.npz")
all_issame = np.load(emb_dir / "all_issame.npy")
dataname = np.load(emb_dir / "dataname.npy")

In [24]:
pred["embs"][0] @ pred["embs"][1], pred["embs"][0] @ pred["embs"][12]

(0.63323724, -0.07951261)

In [18]:
all_issame[:2]

array([1, 1])

In [11]:
len(all_issame)

62000

In [3]:
prepare_data(data_root, val_data_path, "five_mem_concat")

laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile


In [5]:
len(five_ds)

62000

In [8]:
five_ds[0][0].shape

torch.Size([3, 112, 112])